#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from statistics import mode
import math
from collections import Counter
import numpy as np
import sklearn.metrics
import random

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [2]:
def entropy_func(class_count, num_samples): #+
    if num_samples == 0 or class_count == 0:
      return 0
    p = class_count/num_samples    
    return - p * np.log2(p)

class Group: #+
    def __init__(self, group_classes): #target 
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size # [0, 0, 2, 2, 1, 2]

    def count_classes(self):
        class_count = [0, 0, 0] 
        for i in self.group_classes:
          if(i == 0):
            class_count[0] += 1
          elif(i == 1):
            class_count[1] += 1
          elif(i == 2):
            class_count[2] += 1
        return class_count

    def group_entropy(self):
        class_count = self.count_classes()
        k = 0
        sum = 0
        while (k < len(class_count)):
          sum += entropy_func(class_count[k], len(self))
          k += 1
        return sum


class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if self.val is None:
            if data[self.split_feature] < self.split_val:
                return self.child_node_a.predict(data)
            else:
                return self.child_node_b.predict(data)
        else:
            return self.val

class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a, group_b):
        return (group_a.entropy * len(group_a) + group_b.entropy * len(group_b)) / (len(group_a) + len(group_b))

    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        return parent_group.entropy - self.get_split_entropy(child_group_a, child_group_b)

    def get_feature_values(self, data, which_feature):
      feature_values = np.transpose(data)
      return feature_values[which_feature]

    def group_feature_values(self, feature_values, divider, classes): 
      group_child_a = []
      group_child_b = []
      for val, gr in zip(feature_values, classes):
        if val < divider:
            group_child_a.append(gr)
        else:
            group_child_b.append(gr)
      return group_child_a, group_child_b

    def get_best_feature_split(self, data, which_feature, classes): #find best divider value, attribute
        feature_values = self.get_feature_values(data, which_feature)

        max_inf_gain = 0
        div = 0
        for x in feature_values:
            (group_child_a, group_child_b) = self.group_feature_values(feature_values, x, classes) #dostajemy pogrupowane wartosci klas ze wzgledu na divider dla probki 
            divider_inf_gain = self.get_information_gain(Group(np.array(classes)), Group(np.array(group_child_a)), Group(np.array(group_child_b)))
            if divider_inf_gain > max_inf_gain:
              max_inf_gain = divider_inf_gain
              div = x
        return div, max_inf_gain

    def get_best_split(self, data, classes): #return index feature with max infromation gain
        max_inf_gain = 0
        best_divider = 0
        best_feature = 0

        for k in range(len(np.transpose(data))): # dla kazdego atrybuty sprawdz ->>
          divider, inf_gain = self.get_best_feature_split(data, k, classes)
          if inf_gain > max_inf_gain:
            best_feature = k
            max_inf_gain = inf_gain
            best_divider = divider
        return best_divider, best_feature

    def get_data_grouped(self, data, classes, which_feature, divider):
      smaller_data = np.empty((0, 4), float)
      smaller_classes = np.empty((0, 0), int)
      bigger_data = np.empty((0, 4), float)
      bigger_classes = np.empty((0, 0), int)

      feature_values = self.get_feature_values(data, which_feature)
      for ind, x in enumerate(feature_values):
          if x < divider:
                  smaller_data = np.vstack([smaller_data, data[ind]])
                  smaller_classes = np.append(smaller_classes, classes[ind])
          else:
                  bigger_data = np.vstack([bigger_data, data[ind]])
                  bigger_classes = np.append(bigger_classes, classes[ind])
      return (smaller_data, smaller_classes, bigger_data, bigger_classes)

    def build_tree(self, data, classes, depth=0):  #data #target_vlues
        data = np.array(data)
        classes = np.array(classes)

        if self.max_depth == depth:
          classes_group = Group(classes)
          max_howMany = 0
          max_class = 0
          for id, howMany in enumerate(classes_group.count_classes()):
              if howMany > max_howMany:
                max_class = id
                max_howMany = howMany
          return Node(None, None, depth, None, None, max_class)

        if np.unique(classes).shape[0] == 1:
          return Node(None, None, depth, None, None, classes[0])

        divider, feature_id = self.get_best_split(data, classes)
        child_a_data, child_a_classes, child_b_data, child_b_classes = self.get_data_grouped(data, classes, feature_id, divider)

        child_node_a = self.build_tree(child_a_data, child_a_classes, depth + 1)
        child_node_b = self.build_tree(child_b_data, child_b_classes, depth + 1)

        if depth == 0:
            self.tree = Node(feature_id, divider, depth, child_node_a, child_node_b)
            
        else:
            return Node(feature_id, divider, depth, child_node_a, child_node_b)

    def how_works(self, data, classes, depth=0):
        print(depth)
        data = np.array(data)
        classes = np.array(classes)

        if self.max_depth == depth: #when depth is max, wygrywa klasa wiekszosciowa
          classes_group = Group(classes)
          max_howMany = 0
          max_class = 0
          for id, howMany in enumerate(classes_group.count_classes()):
              if howMany > max_howMany:
                max_class = id
                max_howMany = howMany
          print(f"max depth: {max_class}")
          return Node(None, None, depth, None, None, max_class)

        if np.unique(classes).shape[0] == 1: # when node is lisciem
          print(f"lisc: {classes[0]}")
          return Node(None, None, depth, None, None, classes[0])

        #to nie koniec, wiec szukamy dalej wchodzimy dalej w drzewo
        divider, feature_id = self.get_best_split(data, classes)
        child_a_data, child_a_classes, child_b_data, child_b_classes = self.get_data_grouped(data, classes, feature_id, divider)

        print("child node a depth + 1")
        child_node_a = self.how_works(child_a_data, child_a_classes, depth + 1)
        print("child node b depth + 1")
        child_node_b = self.how_works(child_b_data, child_b_classes, depth + 1)

        if depth == 0: # when we start 
            print("depth == 0")
            self.tree = Node(feature_id, divider, depth, child_node_a, child_node_b)
            
        else: #zaden z tych przypadkow, czyli ani nie lisc, ani nie doszlismy do konca max_depth, ani nie start, czyli wychodzenie z zaglebienia np lewostronnego i dalsze poszukiwania aby wyliczcy wszystkie bloki 
            print("pozostaly przypadek")
            return Node(feature_id, divider, depth, child_node_a, child_node_b) 
            
    def predict(self, data):
        return self.tree.predict(data)

### Wyniki obliczen na podanym zbiorze
Dla depth == 3

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)
dc = DecisionTreeClassifier(3)
prediction_values = []
print("start building tree")
dc.build_tree(x_train, y_train)
#dc.how_works(x_train, y_train)
print("end building tree")

for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    print(sample, prediction, gt)
    prediction_values.append(prediction)
prediction_values = np.array(prediction_values)


# Model Accuracy, how often is the classifier correct?
print(prediction_values)
print(y_test)

score_pred = sklearn.metrics.accuracy_score(y_test, prediction_values)
print(f"Accuracy: {score_pred}")


start building tree
end building tree
[6.3 2.5 4.9 1.5] 2 1
[6.8 3.  5.5 2.1] 2 2
[6.4 2.8 5.6 2.2] 2 2
[5.6 3.  4.1 1.3] 1 1
[4.9 3.6 1.4 0.1] 0 0
[6.  3.  4.8 1.8] 2 2
[6.3 2.3 4.4 1.3] 1 1
[4.4 3.2 1.3 0.2] 0 0
[4.4 2.9 1.4 0.2] 0 0
[5.5 2.6 4.4 1.2] 1 1
[6.9 3.1 5.1 2.3] 2 2
[5.5 4.2 1.4 0.2] 0 0
[5.2 2.7 3.9 1.4] 1 1
[6.5 3.  5.5 1.8] 2 2
[7.7 3.  6.1 2.3] 2 2
[2 2 2 1 0 2 1 0 0 1 2 0 1 2 2]
[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2]
Accuracy: 0.9333333333333333


### Wyniki obliczen dla depth == 2

Obserwujemy tutaj juz spadek jakości drzewa decyzyjnego. Nie spada znacząco, ponieważ tak na prawdę zostaje wyliczona nieprawdiłowa wartość tylko dla jednego liścia.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)
dc = DecisionTreeClassifier(2)
prediction_values = []
dc.build_tree(x_train, y_train)

for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    print(sample, prediction, gt)
    prediction_values.append(prediction)
prediction_values = np.array(prediction_values)


# Model Accuracy, how often is the classifier correct?
print(prediction_values)
print(y_test)

score_pred = sklearn.metrics.accuracy_score(y_test, prediction_values)
print(f"Accuracy: {score_pred}")


[6.3 2.5 4.9 1.5] 2 1
[6.8 3.  5.5 2.1] 2 2
[6.4 2.8 5.6 2.2] 2 2
[5.6 3.  4.1 1.3] 1 1
[4.9 3.6 1.4 0.1] 0 0
[6.  3.  4.8 1.8] 1 2
[6.3 2.3 4.4 1.3] 1 1
[4.4 3.2 1.3 0.2] 0 0
[4.4 2.9 1.4 0.2] 0 0
[5.5 2.6 4.4 1.2] 1 1
[6.9 3.1 5.1 2.3] 2 2
[5.5 4.2 1.4 0.2] 0 0
[5.2 2.7 3.9 1.4] 1 1
[6.5 3.  5.5 1.8] 2 2
[7.7 3.  6.1 2.3] 2 2
[2 2 2 1 0 1 1 0 0 1 2 0 1 2 2]
[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2]
Accuracy: 0.8666666666666667


### Wyniki obliczen dla podwyzszonego random_state
Inne dane, a raczej wybrane inaczej, poniważ właśnie to definiuje random_state. Algorytm dalej jest skuteczny, co pokazuje ponad 80% Accurency.

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=574) 
dc = DecisionTreeClassifier(3)
prediction_values = []
dc.build_tree(x_train, y_train)

for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    print(sample, prediction, gt)
    prediction_values.append(prediction)
prediction_values = np.array(prediction_values)


# Model Accuracy, how often is the classifier correct?
print(prediction_values)
print(y_test)

score_pred = sklearn.metrics.accuracy_score(y_test, prediction_values)
print(f"Accuracy: {score_pred}")


[6.7 3.1 4.4 1.4] 1 1
[4.8 3.1 1.6 0.2] 0 0
[6.1 2.8 4.7 1.2] 1 1
[4.8 3.4 1.9 0.2] 0 0
[5.9 3.2 4.8 1.8] 2 1
[6.3 2.5 5.  1.9] 2 2
[6.8 3.  5.5 2.1] 2 2
[5.4 3.7 1.5 0.2] 0 0
[6.4 2.8 5.6 2.1] 2 2
[6.5 3.2 5.1 2. ] 2 2
[6.4 2.9 4.3 1.3] 1 1
[7.7 3.8 6.7 2.2] 2 2
[6.3 2.7 4.9 1.8] 2 2
[6.7 3.  5.  1.7] 2 1
[6.3 2.3 4.4 1.3] 1 1
[1 0 1 0 2 2 2 0 2 2 1 2 2 2 1]
[1 0 1 0 1 2 2 0 2 2 1 2 2 1 1]
Accuracy: 0.8666666666666667


### Wyniki obliczen dla podwyzszonego test_size = 0.25, a wiec mniejsza ilosc zbioru zostala przebadana
Algorytm zmniejszył swoją skuteczność. Jest to spowodowane mniejszą ilością danych, które dostało drzewo decyzyjne, aby nauczyć się jak powinnien klasyfikować próbki. Random_state pozostawiłam na tym samym poziomie, aby móc porównać wydajność algorytmu. 

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)
errors_count = 0
dc = DecisionTreeClassifier(3)
prediction_values = []
dc.build_tree(x_train, y_train)

for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    if prediction != gt:
        errors_count += 1
    print(sample, prediction, gt)
    prediction_values.append(prediction)
print(f"Errors count: {errors_count}")
prediction_values = np.array(prediction_values)


# Model Accuracy, how often is the classifier correct?
print(prediction_values)
print(y_test)

score_pred = sklearn.metrics.accuracy_score(y_test, prediction_values)
print(f"Accuracy: {score_pred}")

[6.3 2.5 4.9 1.5] 1 1
[6.8 3.  5.5 2.1] 2 2
[6.4 2.8 5.6 2.2] 2 2
[5.6 3.  4.1 1.3] 1 1
[4.9 3.6 1.4 0.1] 0 0
[6.  3.  4.8 1.8] 1 2
[6.3 2.3 4.4 1.3] 1 1
[4.4 3.2 1.3 0.2] 0 0
[4.4 2.9 1.4 0.2] 0 0
[5.5 2.6 4.4 1.2] 1 1
[6.9 3.1 5.1 2.3] 2 2
[5.5 4.2 1.4 0.2] 0 0
[5.2 2.7 3.9 1.4] 1 1
[6.5 3.  5.5 1.8] 2 2
[7.7 3.  6.1 2.3] 2 2
[6.5 3.  5.8 2.2] 2 2
[5.5 3.5 1.3 0.2] 0 0
[4.3 3.  1.1 0.1] 0 0
[6.1 2.9 4.7 1.4] 1 1
[4.8 3.  1.4 0.3] 0 0
[5.2 3.4 1.4 0.2] 0 0
[6.3 2.8 5.1 1.5] 1 2
[4.8 3.4 1.9 0.2] 0 0
[6.1 3.  4.9 1.8] 2 2
[5.1 3.8 1.6 0.2] 0 0
[5.4 3.4 1.7 0.2] 0 0
[5.4 3.4 1.5 0.4] 0 0
[5.6 2.8 4.9 2. ] 2 2
[7.7 3.8 6.7 2.2] 2 2
[5.  3.6 1.4 0.2] 0 0
[7.4 2.8 6.1 1.9] 2 2
[6.  2.2 5.  1.5] 1 2
[4.7 3.2 1.6 0.2] 0 0
[5.1 3.5 1.4 0.2] 0 0
[6.  2.2 4.  1. ] 1 1
[5.  2.3 3.3 1. ] 1 1
[7.9 3.8 6.4 2. ] 2 2
[5.4 3.9 1.7 0.4] 0 0
Errors count: 3
[1 2 2 1 0 1 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 1 0 2 0 0 0 2 2 0 2 1 0 0 1 1 2
 0]
[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0 2 2 0 0 